# Open Source Model Conversion

Deep learning models are trained and made available for public consumption using a variety of software packages.  You may want to use model _A_ to solve (or help solve) your problem, but find that it is only available in software package(s) that you don't use.  Your challenge is to convert the existing deep learning model for use with your software package. This document provides a short tutorial on how to convert an open source model to a SAS Viya model, using the SAS Deep Learning Python (DLPy) API.  Keras will be used in this tutorial, but the conversion process is similar for any supported open source package.

## Background

The terms _client computer_ (or _client_ ) and _server computer_ (or _server_ ) will be used throughout the narrative.  These terms relate to the SAS Viya client / server architecture, and are defined as follows:

**CLIENT** - the computer that runs your Jupyter notebook or Python interpreter.

**SERVER** - the computer that hosts your SAS Viya server.
  
Both the client and the server computers must access one or more files as part of the model conversion process. When accessing files, it is important to follow these guidelines:

* Any file references that you specify must be consistent with the operating system of the client (or server) that is hosting the file. For example, file path references on a Linux server and file path references on a Windows client must each use the respective operating system path specification syntax. 
* Any files that you want to access must be visible to the client (or server) computer.

## Overview

SAS DLPy provides several functions that streamline converting open source Keras models into SAS Viya models.  _Convert_ means that your Keras model is automatically translated into a SAS Viya equivalent deep learning model. A flow diagram of the model conversion process is shown below.
<br>

<img src="https://github.com/sassoftware/python-dlpy/raw/master/doc/images/import_keras_model.png" alt="drawing" width="900">

<br>
All of the operations to the left of the dashed red line involve the definition, training, and storage of a Keras model.  You or someone else has already defined and trained a Keras model. The task is to import the trained Keras model to SAS Viya.  

All of the operations located on the right side of the dashed red line are steps that are required to build a SAS Viya model.  The processing blocks that straddle the dashed red line represent the SAS DLPy conversion utilities that create the SAS Viya version of your Keras deep learning model.

Model conversion using SAS DLPy is a two-step procedure.  The first step translates the Keras model definition into an equivalent SAS Viya model definition, using the *from_keras_model()* function. 

The Python code below illustrates using the *from_keras_model()* function to translate the Keras definition of a simple LeNet model into a SAS Viya model definition.  The code is a snippet that is excerpted from a complete end-to-end [LeNet model conversion example](https://github.com/sassoftware/python-dlpy/blob/master/examples/keras_model_conversion/Import_Keras_LeNet.ipynb) . Interested readers should examine the entire example.
<br>
<br>
```python
import os

model_name = 'lenet'
# NOTE: Keras model weight file resides on client computer
model1,use_gpu = Model.from_keras_model(conn=s, 
                                        keras_model=model, 
                                        output_model_table=model_name,
                                        include_weights=True,                                      
                                        input_weights_file=os.path.join(client_image_root,
                                                                        "lenet.h5")  
                                        )
```
<br>
<br>
An HDF5 file is generated as part of the first step <em>if</em> you plan to import pre-trained weights to your Viya model.  The HDF5 file name is created by concatenating the <strong>model_name</strong> with the string <em>_weights.kerasmodel.h5</em>. Here the HDF5 file is named <em>lenet_weights.kerasmodel.h5</em>).


The second (and optional) step of model conversion uses the SAS DLPy *load_weights()* function to import the Keras model weights as part of your SAS Viya model.  The code for this step is shown below for the same [LeNet model conversion example](https://github.com/sassoftware/python-dlpy/blob/master/examples/keras_model_conversion/Import_Keras_LeNet.ipynb) .
<br>
<br>
```python
# NOTE 1: model weights file (lenet_weights.kerasmodel.h5) resides on the server
# NOTE 2: image labels file (mnist_labels.csv) resides on the client
model1.load_weights(path=server_image_root+"/lenet_weights.kerasmodel.h5", 
                    labels=True,                   
                    label_file_name=os.path.join(os.getcwd(),              
                                                 "mnist_labels.csv"),       
                    label_length=1)
```
<br>
<br>
Most of the model conversion process occurs on the client.  Note that even though the HDF5 file that is generated during the first conversion step is stored locally on the client computer, the HDF5 file must ultimately be processed by the server. <strong>Unless your client and server machines share a common file system, you must copy the HDF5 file from the client to a location on the server.</strong>

## Supported Open Source Packages and Layers

SAS DLPy can perform automatic model conversions for Keras, ONNX, and Caffe models as long as the model in question consists  of the following layers:

- <strong>Keras</strong>
  - Activation
  - Add
  - AveragePooling2D
  - BatchNormalization
  - Concatenate
  - Conv2D
  - CudnnGRU
  - CudnnLSTM
  - Dense
  - Dropout
  - Flatten
  - GlobalAveragePooling2D
  - GRU
  - InputLayer
  - Lambda
  - LSTM
  - MaxPooling2D
  - SimpleRNN
  - ZeroPadding2D
<br>
<br>
- <strong>ONNX</strong> (<em>note: these are node operations rather than layer types</em>)
  - Add
  - AveragePool
  - BatchNormalization
  - Concat
  - Conv
  - Gemm
  - GlobalAveragePool
  - MatMul
  - MaxPool
  - Sum
<br>
<br>
- <strong>Caffe</strong>
  - Batchnorm
  - Convolution
  - Data
  - Eltwise
  - Inner Product
  - Memory Data
  - Pooling  


If your open source package is not listed above, or your model includes layers that are not shown, the model conversion will fail.  If you experience problems importing models, or if you have a request to expand the list of open source packages and/or model layers, please open an issue on the SAS DLPy [issues forum](https://github.com/sassoftware/python-dlpy/issues) .